# Otimização de Portifólio de Investimentos com algoritmo genético

**TechChallenge IA para Devs - PósTech FIAP** >>
Grupo 18


O primeiro trabalho foi criar um dataframe pertinente.
Buscamos uma lista de ativos que representasse diferentes perfis de investidores, mas que fossem acessíveis ainda para amadores.

Em seguida, buscamos as tabelas de rentabilidades de bancos como Itaú, Banco do Brasil e Bradesco, as tabelas do tesouro direto SELIC, PréFixado, IPCA em pdf. Com a ajuda do ChatGPT consolidamos esses dados. Ademais, com a API da yfinance, buscamos os dados históricos para conseguirmos integrar o cálculo de rentabilidades de 12, 24 e 36 meses para o dólar, as principais criptos e 11 ações brasileiras iniciais.


Necessitamos calcular o ROI e a TAXA de RISCO ponderada para chegar no melhor SHARPE RATIO, métrica que elegemos para avaliar nosso portifólio.

Esses 34 ativos compuseram nosso GENOMA INICIAL com uma alocação de recursos hipotética.


O Sharpe Ratio mede o desempenho ajustado ao risco de um investimento ou portfólio. Ele indica quanto retorno adicional você está recebendo para cada unidade de risco que você está assumindo.

***Interpretação de Valores do Sharpe Ratio:***

  🔴Sharpe Ratio baixo (< 1): Retorno baixo para o nível de risco. O investimento está assumindo muito risco para o retorno que gera.

  🟡Sharpe Ratio de 1 a 2: Relacionamento razoável entre risco e retorno. O investimento está compensando o risco de forma adequada.

  🟢Sharpe Ratio de 2 a 3: Excelente retorno ajustado ao risco.

  🔵Sharpe Ratio > 3: Desempenho excepcional, onde o retorno para o risco assumido é muito alto.


  PIPELINE:


1.   Geração do dataframe
2.   EAP - Análise dos Dados
3.   Algorítimo Genético
      * 1.   População Inicial: Criamos uma população inicial de portfólios, genoma.
      * 2.   Função de Fitness: Avaliaremos cada portfólio usando a relação de Sharpe (retorno vs. risco).
      * 3.   Seleção: Selecionaremos os melhores portfólios com base na função de fitness.
      * 4.   Cruzamento e Mutação: Cruzamos os melhores portfólios para gerar novos e aplicamos mutações.
      * 5.   Critério de Parada: Rodaremos o algoritmo por um número fixo de gerações ou até que não haja mais melhoria significativa.

      OBS: Aplicaremos soluções para avaliar e restringir que nenhuma população criada (portfólio) seja inferior ou superior a 100%

      OBS: Aplicaremos o elitismo, para selecionar os melhores portfólios a cada geração.

      OBS: Aplicaremos o Streamlit

In [19]:
# Vamos analisar nosso pool inicial
import pandas as pd

# Leitura do arquivo CSV
df = pd.read_csv('//content/Pool_Investimentos.csv')

# Mostrar as primeiras 5 linhas do DataFrame
print("Primeiras 5 linhas do DataFrame:")
print(df.head())

# Informações básicas sobre o DataFrame (inclui tipos de dados e nulos)
print("\nInformações básicas (EAP) do DataFrame:")
print(df.info())

# Contagem de valores nulos
print("\nContagem de valores nulos por coluna:")
print(df.isnull().sum())

Primeiras 5 linhas do DataFrame:
               Ativo  Rentabilidade 12 meses  Rentabilidade 24 meses  \
0  Tesouro Prefixado                    7.90                   22.00   
1      Tesouro RendA                   10.11                   14.18   
2      Tesouro Selic                    7.71                   10.92   
3       Tesouro IPCA                    4.83                    7.72   
4            Bitcoin                   40.20                   90.40   

   Rentabilidade 36 meses   ROI  Taxa de Risco  
0                   30.50  1.12           0.25  
1                   27.92  1.20           0.20  
2                   25.66  1.15           0.30  
3                   18.94  1.07           0.35  
4                  120.40  2.00           0.50  

Informações básicas (EAP) do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  -------------- 

In [20]:
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função para gerar o genoma inicial de portfólios com 34 ativos
#def gerar_portfolios(num_portfolios, num_ativos):
    #return [np.random.dirichlet(np.ones(num_ativos)) for _ in range(num_portfolios)] (forma randomizada de gerar genoma)

genoma_inicial = np.array([
    0.03,  # Tesouro Prefixado
    0.05,  # Tesouro RendA
    0.03,  # Tesouro Selic
    0.02,  # Tesouro IPCA
    0.06,  # Bitcoin
    0.01,  # Cardano
    0.05,  # Ethereum
    0.02,  # Litecoin
    0.03,  # Dólar
    0.03,  # VALE3.SA
    0.02,  # PETR4.SA
    0.03,  # JBSS3.SA
    0.04,  # MGLU3.SA
    0.03,  # RENT3.SA
    0.04,  # B3SA3.SA
    0.02,  # WEGE3.SA
    0.02,  # EMBR3.SA
    0.03,  # GOLL4.SA
    0.04,  # ITUB4.SA
    0.03,  # Renda Fixa BB 1
    0.02,  # Renda Fixa BB 2
    0.03,  # Renda Fixa BB 3
    0.02,  # Renda Fixa BB 4
    0.03,  # Renda Fixa BB 5
    0.02,  # Renda Fixa Bradesco 1
    0.01,  # Renda Fixa Bradesco 2
    0.03,  # Renda Fixa Bradesco 3
    0.03,  # Renda Fixa Bradesco 4
    0.03,  # Renda Fixa Bradesco 5
    0.03,  # Renda Fixa Itaú 1
    0.03,  # Renda Fixa Itaú 2
    0.03,  # Renda Fixa Itaú 3
    0.03,  # Renda Fixa Itaú 4
    0.03   # Renda Fixa Itaú 5
])


# Verificando se a soma das alocações é 100%
assert np.isclose(genoma_inicial.sum(), 1.0), "As alocações devem somar 100% (ou 1.0 em fração)"

# Função para gerar a população inicial com o genoma inicial fixo
def gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, num_ativos):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(num_ativos)))
    return populacao

# Função para rodar o algoritmo genético com genoma inicial fixo
def algoritmo_genetico_com_genoma_inicial(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=50):
    populacao = gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, len(retornos))
    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Verificar se algum portfólio é inválido
        for portfolio in populacao:
            assert np.isclose(portfolio.sum(), 1.0), "Portfólio inválido: soma das alocações não é 100%"


        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = cruzamento(pai1, pai2)

            #Verificação e Normalização do Portfólio após Crossover
            #Normalizar os portfólios resultantes (filhos) para garantir que a soma das alocações seja 1 (ou 100%)
            filho1 = filho1 / filho1.sum()
            filho2 = filho2 / filho2.sum()

            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

def cruzamento(pai1, pai2):
    ponto_corte = np.random.randint(1, len(pai1) - 1)
    filho1 = np.concatenate((pai1[:ponto_corte], pai2[ponto_corte:]))
    filho2 = np.concatenate((pai2[:ponto_corte], pai1[ponto_corte:]))
    return filho1, filho2

def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)

        #Verificação e Normalização do Portfólio após Mutação
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas entre 0 e 100%
        portfolio /= portfolio.sum()  # Normalizar para que a soma seja exatamente 1 (ou 100%)
    return portfolio

# Exemplo de dados reais para retornos e riscos
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com o genoma inicial fixo
melhor_portfolio = algoritmo_genetico_com_genoma_inicial(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Exibir a distribuição ideal do investimento
print(distribuicao_df)

[*********************100%***********************]  15 of 15 completed
<ipython-input-20-0bac084154e9>:22: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()


Geracao 1, Melhor Sharpe Ratio: 0.18872035578557467
Geracao 2, Melhor Sharpe Ratio: 0.3183357100802153
Geracao 3, Melhor Sharpe Ratio: 0.3998547028626909
Geracao 4, Melhor Sharpe Ratio: 0.5814446867398019
Geracao 5, Melhor Sharpe Ratio: 0.5943030015324852
Geracao 6, Melhor Sharpe Ratio: 0.6916438254062569
Geracao 7, Melhor Sharpe Ratio: 0.6940471347671091
Geracao 8, Melhor Sharpe Ratio: 0.7110785281161647
Geracao 9, Melhor Sharpe Ratio: 0.7571682689966811
Geracao 10, Melhor Sharpe Ratio: 0.8183547184078847
Geracao 11, Melhor Sharpe Ratio: 0.8209189653596095
Geracao 12, Melhor Sharpe Ratio: 0.9045992585802197
Geracao 13, Melhor Sharpe Ratio: 0.9376729202953481
Geracao 14, Melhor Sharpe Ratio: 0.9765188488549116
Geracao 15, Melhor Sharpe Ratio: 0.9996078267260672
Geracao 16, Melhor Sharpe Ratio: 0.9996078267260672
Geracao 17, Melhor Sharpe Ratio: 1.0100707723399984
Geracao 18, Melhor Sharpe Ratio: 1.0309825740864578
Geracao 19, Melhor Sharpe Ratio: 1.0386905277318952
Geracao 20, Melhor S

In [21]:
#Vamos aumentar a geração para 100

import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função para gerar o genoma inicial de portfólios com 34 ativos
#def gerar_portfolios(num_portfolios, num_ativos):
    #return [np.random.dirichlet(np.ones(num_ativos)) for _ in range(num_portfolios)] (forma randomizada de gerar genoma)

genoma_inicial = np.array([
    0.03,  # Tesouro Prefixado
    0.05,  # Tesouro RendA
    0.03,  # Tesouro Selic
    0.02,  # Tesouro IPCA
    0.06,  # Bitcoin
    0.01,  # Cardano
    0.05,  # Ethereum
    0.02,  # Litecoin
    0.03,  # Dólar
    0.03,  # VALE3.SA
    0.02,  # PETR4.SA
    0.03,  # JBSS3.SA
    0.04,  # MGLU3.SA
    0.03,  # RENT3.SA
    0.04,  # B3SA3.SA
    0.02,  # WEGE3.SA
    0.02,  # EMBR3.SA
    0.03,  # GOLL4.SA
    0.04,  # ITUB4.SA
    0.03,  # Renda Fixa BB 1
    0.02,  # Renda Fixa BB 2
    0.03,  # Renda Fixa BB 3
    0.02,  # Renda Fixa BB 4
    0.03,  # Renda Fixa BB 5
    0.02,  # Renda Fixa Bradesco 1
    0.01,  # Renda Fixa Bradesco 2
    0.03,  # Renda Fixa Bradesco 3
    0.03,  # Renda Fixa Bradesco 4
    0.03,  # Renda Fixa Bradesco 5
    0.03,  # Renda Fixa Itaú 1
    0.03,  # Renda Fixa Itaú 2
    0.03,  # Renda Fixa Itaú 3
    0.03,  # Renda Fixa Itaú 4
    0.03   # Renda Fixa Itaú 5
])


# Verificando se a soma das alocações é 100%
assert np.isclose(genoma_inicial.sum(), 1.0), "As alocações devem somar 100% (ou 1.0 em fração)"

# Função para gerar a população inicial com o genoma inicial fixo
def gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, num_ativos):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(num_ativos)))
    return populacao

# Função para rodar o algoritmo genético com genoma inicial fixo
def algoritmo_genetico_com_genoma_inicial(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100):
    populacao = gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, len(retornos))
    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Verificar se algum portfólio é inválido
        for portfolio in populacao:
            assert np.isclose(portfolio.sum(), 1.0), "Portfólio inválido: soma das alocações não é 100%"


        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = cruzamento(pai1, pai2)

            #Verificação e Normalização do Portfólio após Crossover
            #Normalizar os portfólios resultantes (filhos) para garantir que a soma das alocações seja 1 (ou 100%)
            filho1 = filho1 / filho1.sum()
            filho2 = filho2 / filho2.sum()

            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

def cruzamento(pai1, pai2):
    ponto_corte = np.random.randint(1, len(pai1) - 1)
    filho1 = np.concatenate((pai1[:ponto_corte], pai2[ponto_corte:]))
    filho2 = np.concatenate((pai2[:ponto_corte], pai1[ponto_corte:]))
    return filho1, filho2

def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)

        #Verificação e Normalização do Portfólio após Mutação
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas entre 0 e 100%
        portfolio /= portfolio.sum()  # Normalizar para que a soma seja exatamente 1 (ou 100%)
    return portfolio

# Exemplo de dados reais para retornos e riscos
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com o genoma inicial fixo
melhor_portfolio = algoritmo_genetico_com_genoma_inicial(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Exibir a distribuição ideal do investimento
print(distribuicao_df)

[*********************100%***********************]  15 of 15 completed
<ipython-input-21-d1c978b3a41c>:24: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()


Geracao 1, Melhor Sharpe Ratio: 0.46870117388508065
Geracao 2, Melhor Sharpe Ratio: 0.5474897997314492
Geracao 3, Melhor Sharpe Ratio: 0.736386339882661
Geracao 4, Melhor Sharpe Ratio: 0.736386339882661
Geracao 5, Melhor Sharpe Ratio: 0.8166516751183286
Geracao 6, Melhor Sharpe Ratio: 0.8166516751183286
Geracao 7, Melhor Sharpe Ratio: 0.8166516751183286
Geracao 8, Melhor Sharpe Ratio: 0.8698218332911803
Geracao 9, Melhor Sharpe Ratio: 0.8957519611836512
Geracao 10, Melhor Sharpe Ratio: 0.9855482441966399
Geracao 11, Melhor Sharpe Ratio: 0.9855482441966399
Geracao 12, Melhor Sharpe Ratio: 1.0214790276864996
Geracao 13, Melhor Sharpe Ratio: 1.051663065540511
Geracao 14, Melhor Sharpe Ratio: 1.056312366854034
Geracao 15, Melhor Sharpe Ratio: 1.0758030469650375
Geracao 16, Melhor Sharpe Ratio: 1.0981258173516395
Geracao 17, Melhor Sharpe Ratio: 1.1412248763325694
Geracao 18, Melhor Sharpe Ratio: 1.174521515842255
Geracao 19, Melhor Sharpe Ratio: 1.1780557161241274
Geracao 20, Melhor Sharpe

In [22]:
import pandas as pd
import yfinance as yf

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptomoedas
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Verificar se existem valores NaN nos dados históricos
print("Verificando valores NaN nos dados históricos:")
print(dados_historicos_completos.isna().sum())

# Verificar se existem valores zero nos dados históricos
print("\nVerificando valores zero nos dados históricos:")
print((dados_historicos_completos == 0).sum())


[*********************100%***********************]  15 of 15 completed


Verificando valores NaN nos dados históricos:
Ticker
ADA-USD       0
B3SA3.SA    350
BRL=X       314
BTC-USD       0
EMBR3.SA    350
ETH-USD       0
GOLL4.SA    350
ITUB4.SA    350
JBSS3.SA    350
LTC-USD       0
MGLU3.SA    350
PETR4.SA    350
RENT3.SA    350
VALE3.SA    350
WEGE3.SA    350
dtype: int64

Verificando valores zero nos dados históricos:
Ticker
ADA-USD     0
B3SA3.SA    0
BRL=X       0
BTC-USD     0
EMBR3.SA    0
ETH-USD     0
GOLL4.SA    0
ITUB4.SA    0
JBSS3.SA    0
LTC-USD     0
MGLU3.SA    0
PETR4.SA    0
RENT3.SA    0
VALE3.SA    0
WEGE3.SA    0
dtype: int64


In [23]:
# Preencher os valores NaN com a média dos dados históricos para cada ativo
dados_historicos_preenchidos = dados_historicos_completos.apply(lambda col: col.fillna(col.mean()), axis=0)

# Verificar novamente se ainda existem valores NaN após o preenchimento
print("Verificando valores NaN após preenchimento:")
print(dados_historicos_preenchidos.isna().sum())


Verificando valores NaN após preenchimento:
Ticker
ADA-USD     0
B3SA3.SA    0
BRL=X       0
BTC-USD     0
EMBR3.SA    0
ETH-USD     0
GOLL4.SA    0
ITUB4.SA    0
JBSS3.SA    0
LTC-USD     0
MGLU3.SA    0
PETR4.SA    0
RENT3.SA    0
VALE3.SA    0
WEGE3.SA    0
dtype: int64


In [24]:
#Tratando no código o Nan
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado (deve conter 34 ativos)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN nos dados históricos com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Verificar se ainda existem valores NaN
print("Verificando valores NaN após preenchimento:")
print(dados_historicos_completos.isna().sum())

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função para gerar o genoma inicial de portfólios com 34 ativos
#def gerar_portfolios(num_portfolios, num_ativos):
    #return [np.random.dirichlet(np.ones(num_ativos)) for _ in range(num_portfolios)] (forma randomizada de gerar genoma)
    #queremos um genoma inicial fixo
genoma_inicial = np.array([
    0.03,  # Tesouro Prefixado
    0.05,  # Tesouro RendA
    0.03,  # Tesouro Selic
    0.02,  # Tesouro IPCA
    0.06,  # Bitcoin
    0.01,  # Cardano
    0.05,  # Ethereum
    0.02,  # Litecoin
    0.03,  # Dólar
    0.03,  # VALE3.SA
    0.02,  # PETR4.SA
    0.03,  # JBSS3.SA
    0.04,  # MGLU3.SA
    0.03,  # RENT3.SA
    0.04,  # B3SA3.SA
    0.02,  # WEGE3.SA
    0.02,  # EMBR3.SA
    0.03,  # GOLL4.SA
    0.04,  # ITUB4.SA
    0.03,  # Renda Fixa BB 1
    0.02,  # Renda Fixa BB 2
    0.03,  # Renda Fixa BB 3
    0.02,  # Renda Fixa BB 4
    0.03,  # Renda Fixa BB 5
    0.02,  # Renda Fixa Bradesco 1
    0.01,  # Renda Fixa Bradesco 2
    0.03,  # Renda Fixa Bradesco 3
    0.03,  # Renda Fixa Bradesco 4
    0.03,  # Renda Fixa Bradesco 5
    0.03,  # Renda Fixa Itaú 1
    0.03,  # Renda Fixa Itaú 2
    0.03,  # Renda Fixa Itaú 3
    0.03,  # Renda Fixa Itaú 4
    0.03   # Renda Fixa Itaú 5
])


# Verificando se a soma das alocações é 100%
assert np.isclose(genoma_inicial.sum(), 1.0), "As alocações devem somar 100% (ou 1.0 em fração)"

# Função para gerar a população inicial com o genoma inicial fixo
def gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, num_ativos):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(num_ativos)))
    return populacao

# Função para rodar o algoritmo genético com genoma inicial fixo
def algoritmo_genetico_com_genoma_inicial(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100):
    populacao = gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, len(retornos))
    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # # Verificar se algum portfólio é inválido
        for portfolio in populacao:
            assert np.isclose(portfolio.sum(), 1.0), "Portfólio inválido: soma das alocações não é 100%"



        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = cruzamento(pai1, pai2)

            # Normalizar os filhos
            filho1 = filho1 / filho1.sum()
            filho2 = filho2 / filho2.sum()

            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

def cruzamento(pai1, pai2):
    ponto_corte = np.random.randint(1, len(pai1) - 1)
    filho1 = np.concatenate((pai1[:ponto_corte], pai2[ponto_corte:]))
    filho2 = np.concatenate((pai2[:ponto_corte], pai1[ponto_corte:]))
    return filho1, filho2

def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas para garantir o portfólio com soma =1 ou 100%
        portfolio /= portfolio.sum()  # Normalizar
    return portfolio

# Exemplo de dados reais para retornos e riscos
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com o genoma inicial fixo
melhor_portfolio = algoritmo_genetico_com_genoma_inicial(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Exibir a distribuição ideal do investimento
print(distribuicao_df)

[*********************100%***********************]  15 of 15 completed


Verificando valores NaN após preenchimento:
Ticker
ADA-USD     0
B3SA3.SA    0
BRL=X       0
BTC-USD     0
EMBR3.SA    0
ETH-USD     0
GOLL4.SA    0
ITUB4.SA    0
JBSS3.SA    0
LTC-USD     0
MGLU3.SA    0
PETR4.SA    0
RENT3.SA    0
VALE3.SA    0
WEGE3.SA    0
dtype: int64
Geracao 1, Melhor Sharpe Ratio: 0.628894290931849
Geracao 2, Melhor Sharpe Ratio: 0.6288942909318496
Geracao 3, Melhor Sharpe Ratio: 0.6663379244328498
Geracao 4, Melhor Sharpe Ratio: 0.6796447247853412
Geracao 5, Melhor Sharpe Ratio: 0.7965726988756365
Geracao 6, Melhor Sharpe Ratio: 0.8681109137675176
Geracao 7, Melhor Sharpe Ratio: 0.9095001439837054
Geracao 8, Melhor Sharpe Ratio: 0.912911640758323
Geracao 9, Melhor Sharpe Ratio: 0.9276437011739856
Geracao 10, Melhor Sharpe Ratio: 0.9384293135609882
Geracao 11, Melhor Sharpe Ratio: 0.9543600531496627
Geracao 12, Melhor Sharpe Ratio: 0.967083114838261
Geracao 13, Melhor Sharpe Ratio: 0.9905483377489884
Geracao 14, Melhor Sharpe Ratio: 1.0019503861330532
Geracao 15

In [2]:
#Vamos simular um modelo de negócio, onde nosso cliente tem um pool de investimento inicial
#Na nossa consultoria oferecemos 34 ativos para diversificar
#Vamos trocar o genoma inicial para simular o investimento do nosso cliente

#Tratando no código o Nan
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado (deve conter 34 ativos)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN nos dados históricos com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Verificar se ainda existem valores NaN
print("Verificando valores NaN após preenchimento:")
print(dados_historicos_completos.isna().sum())

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função para gerar o genoma inicial de portfólios com 34 ativos
#def gerar_portfolios(num_portfolios, num_ativos):
    #return [np.random.dirichlet(np.ones(num_ativos)) for _ in range(num_portfolios)] (forma randomizada de gerar genoma)

genoma_inicial = np.array([
    0.00,  # Tesouro Prefixado (sem alocação)
    0.00,  # Tesouro RendA (sem alocação)
    0.20,  # Tesouro Selic (20% do portfólio)
    0.00,  # Tesouro IPCA (sem alocação)
    0.05,  # Bitcoin (5% do portfólio)
    0.00,  # Cardano (sem alocação)
    0.03,  # Ethereum (5% do portfólio)
    0.00,  # Litecoin (sem alocação)
    0.00,  # Dólar (sem alocação)
    0.03,  # VALE3.SA (5% do portfólio)
    0.05,  # PETR4.SA (5% do portfólio)
    0.00,  # JBSS3.SA (sem alocação)
    0.00,  # MGLU3.SA (sem alocação)
    0.00,  # RENT3.SA (sem alocação)
    0.00,  # B3SA3.SA (sem alocação)
    0.00,  # WEGE3.SA (sem alocação)
    0.00,  # EMBR3.SA (sem alocação)
    0.05,  # GOLL4.SA (5% do portfólio)
    0.05,  # ITUB4.SA (5% do portfólio)
    0.06,  # Renda Fixa BB 1 (10% do portfólio)
    0.10,  # Renda Fixa BB 2 (10% do portfólio)
    0.00,  # Renda Fixa BB 3 (sem alocação)
    0.00,  # Renda Fixa BB 4 (sem alocação)
    0.00,  # Renda Fixa BB 5 (sem alocação)
    0.05,  # Renda Fixa Bradesco 1 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 2 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 3 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 4 (5% do portfólio)
    0.00,  # Renda Fixa Bradesco 5 (sem alocação)
    0.05,  # Renda Fixa Itaú 1 (5% do portfólio)
    0.05,  # Renda Fixa Itaú 2 (5% do portfólio)
    0.03,  # Renda Fixa Itaú 3 (5% do portfólio)
    0.05,  # Renda Fixa Itaú 4 (5% do portfólio)
    0.00   # Renda Fixa Itaú 5 (sem alocação)
])


# Verificando se a soma das alocações é 100%
assert np.isclose(genoma_inicial.sum(), 1.0), "As alocações devem somar 100% (ou 1.0 em fração)"

# Função para gerar a população inicial com o genoma inicial fixo
def gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, num_ativos):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(num_ativos)))
    return populacao

# Função para rodar o algoritmo genético com genoma inicial fixo
def algoritmo_genetico_com_genoma_inicial(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100):
    populacao = gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, len(retornos))
    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):

        # Calcular o Sharpe Ratio (fitness) para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Verificar se algum portfólio é inválido
        for portfolio in populacao:
            assert np.isclose(portfolio.sum(), 1.0), "Portfólio inválido: soma das alocações não é 100%"

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = cruzamento(pai1, pai2)

            # Normalizar os filhos
            filho1 = filho1 / filho1.sum()  # Normalizar para garantir soma 100%
            filho2 = filho2 / filho2.sum()  # Normalizar para garantir soma 100%

            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

    # função de cruzamento de ponto único - Crossover
def cruzamento(pai1, pai2):
    ponto_corte = np.random.randint(1, len(pai1) - 1)
    filho1 = np.concatenate((pai1[:ponto_corte], pai2[ponto_corte:]))
    filho2 = np.concatenate((pai2[:ponto_corte], pai1[ponto_corte:]))
    return filho1, filho2

def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas entre 0 e 100%
        portfolio /= portfolio.sum()  # Normalizar para que a soma seja exatamente 1 (ou 100%)
    return portfolio

# Exemplo de dados reais para retornos e riscos
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com o genoma inicial fixo
melhor_portfolio = algoritmo_genetico_com_genoma_inicial(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Função para salvar o DataFrame em um novo CSV
df.to_csv('/content/Pool_Investimentos_Atualizado.csv', index=False)

# Exibir a distribuição ideal do investimento
print(distribuicao_df)

[*********************100%***********************]  15 of 15 completed


Verificando valores NaN após preenchimento:
Ticker
ADA-USD     0
B3SA3.SA    0
BRL=X       0
BTC-USD     0
EMBR3.SA    0
ETH-USD     0
GOLL4.SA    0
ITUB4.SA    0
JBSS3.SA    0
LTC-USD     0
MGLU3.SA    0
PETR4.SA    0
RENT3.SA    0
VALE3.SA    0
WEGE3.SA    0
dtype: int64
Geracao 1, Melhor Sharpe Ratio: 0.19011244330744978
Geracao 2, Melhor Sharpe Ratio: 0.19011244330744978
Geracao 3, Melhor Sharpe Ratio: 0.33024366936602123
Geracao 4, Melhor Sharpe Ratio: 0.41056209874239713
Geracao 5, Melhor Sharpe Ratio: 0.43296561665075967
Geracao 6, Melhor Sharpe Ratio: 0.5242561958410719
Geracao 7, Melhor Sharpe Ratio: 0.5296472418832776
Geracao 8, Melhor Sharpe Ratio: 0.5296472418832776
Geracao 9, Melhor Sharpe Ratio: 0.5708489987594355
Geracao 10, Melhor Sharpe Ratio: 0.5904753260483235
Geracao 11, Melhor Sharpe Ratio: 0.6336176562851422
Geracao 12, Melhor Sharpe Ratio: 0.6488261230813889
Geracao 13, Melhor Sharpe Ratio: 0.6551527767991048
Geracao 14, Melhor Sharpe Ratio: 0.6864260630285092
Ge

# Respondendo a essas perguntas de referência
Qual é a representação da solução (genoma)?

Qual é a função de fitness?

Qual é o método de seleção?

Qual método de crossover você vai implementar?

Qual será o método de inicialização?

Qual o critério de parada?



---

**Representação da solução (genoma):** O genoma é representado por um array de números reais (frações) que somam 1.0 (ou 100%), indicando a alocação percentual de cada ativo no portfólio. Fizemos um array manual para os 34 ativos. No primeiro teste com todos os ativos alocados e no segundo com alguns somente.

**Função de fitness:** A função de fitness é o Sharpe Ratio, que mede o retorno ajustado ao risco do portfólio. Quanto maior o Sharpe Ratio, melhor o portfólio. O objetivo é maximizar o Sharpe Ratio, o que significa encontrar um portfólio com alto retorno esperado e baixo risco (desvio padrão). def calcular_sharpe

**Método de seleção:** O método de seleção utilizado no algoritmo é a seleção por torneio. Neste método, um número fixo de indivíduos (portfólios) é escolhido aleatoriamente da população, e o indivíduo com o melhor fitness (Sharpe Ratio) entre eles é selecionado para reprodução. Esse processo é repetido para gerar a nova população. def selecao_torneio

**Método de crossover:** O método de crossover (cruzamento) implementado é o crossover de ponto único. Um ponto de corte aleatório é escolhido, e as frações do genoma de dois portfólios são trocadas a partir desse ponto para gerar dois novos portfólios. def cruzamento

**Método de inicialização:** A inicialização do algoritmo é feita com uma população de portfólios, onde o primeiro portfólio é o genoma inicial fixo (um portfólio sugerido previamente), e os outros são gerados de forma aleatória usando a distribuição de Dirichlet, que garante que as frações somam 1.0 (100%). def gerar_portfolios_com_genoma_inicial

**O critério de parada:** é o número fixo de gerações. O algoritmo é configurado para rodar por um número determinado de gerações (por exemplo, 100 gerações), e o melhor portfólio ao final dessas gerações é considerado a solução.  def algoritmo_genetico_com_genoma_inicial







# Aplicando as validações no Crossover de Um Ponto

Restringir as alocações para um máximo de 20% em cada ativo

Normalizar e garantir portfólios de 100% após crossover e mutação

Aplicar penalidade no fitness

Aplicar o elitismo


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado (deve conter 34 ativos)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN nos dados históricos com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Verificar se ainda existem valores NaN
print("Verificando valores NaN após preenchimento:")
print(dados_historicos_completos.isna().sum())

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função de penalização para alocação maior que 20%
def penalidade_alocacao(portfolio, max_alocacao=0.2):
    penalidade = np.sum(np.maximum(0, portfolio - max_alocacao))  # Penaliza ativos que ultrapassam 20%
    return penalidade

# Função de fitness com penalidade de alocação
def fitness_com_penalidade(portfolio, retornos, riscos, taxa_livre_risco, penalizacao=0.1):
    sharpe = calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco)
    penalidade = penalidade_alocacao(portfolio)
    return sharpe - penalizacao * penalidade  # Penaliza Sharpe se houver alocações acima de 20%

# Função para gerar a população inicial com o genoma inicial fixo
def gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, num_ativos):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(num_ativos)))
    return populacao

# Função para rodar o algoritmo genético com elitismo e restrição de alocação
def algoritmo_genetico_com_elitismo(retornos, riscos, retornos_12m, retornos_24m, retornos_36m, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100, elitismo_taxa=0.1):
    populacao = gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, len(retornos))
    melhor_portfolio = genoma_inicial
    melhor_sharpe = fitness_com_penalidade(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):

        # Calcular o Sharpe Ratio com penalidade para cada portfólio
        fitness_scores = np.array([fitness_com_penalidade(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Verificar se algum portfólio é inválido
        for portfolio in populacao:
            assert np.isclose(portfolio.sum(), 1.0), "Portfólio inválido: soma das alocações não é 100%"

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Elitismo: preservar os melhores portfólios
        elite_size = int(elitismo_taxa * len(populacao))
        elite = [populacao[i] for i in np.argsort(fitness_scores)[-elite_size:]]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = elite.copy()  # Preservar os melhores

        while len(nova_populacao) < len(populacao):
            pai1, pai2 = np.random.choice(populacao, size=2, replace=False)
            filho1, filho2 = cruzamento(pai1, pai2)

            # Normalizar os filhos
            filho1 = filho1 / filho1.sum()  # Garantir que a soma seja 100%
            filho2 = filho2 / filho2.sum()  # Garantir que a soma seja 100%

            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao[:num_portfolios]  # Garantir o tamanho da população

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    # Cálculo do retorno total em 12, 24 e 36 meses
    retorno_12m = np.dot(melhor_portfolio, retornos_12m)
    retorno_24m = np.dot(melhor_portfolio, retornos_24m)
    retorno_36m = np.dot(melhor_portfolio, retornos_36m)

    print(f"\nMelhor Portfólio: {melhor_portfolio}")
    print(f"Retorno esperado em 12 meses: {retorno_12m * 100:.2f}%")
    print(f"Retorno esperado em 24 meses: {retorno_24m * 100:.2f}%")
    print(f"Retorno esperado em 36 meses: {retorno_36m * 100:.2f}%")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação com restrição de soma de alocações a 100% e elitismo
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

# Função de cruzamento (crossover de um ponto)
def cruzamento(pai1, pai2):
    ponto_corte = np.random.randint(1, len(pai1) - 1)
    filho1 = np.concatenate((pai1[:ponto_corte], pai2[ponto_corte:]))
    filho2 = np.concatenate((pai2[:ponto_corte], pai1[ponto_corte:]))
    return filho1, filho2

# Função de mutação (adiciona uma variação aleatória à alocação de um ativo)
def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas entre 0 e 100%
        portfolio /= portfolio.sum()  # Normalizar para que a soma seja exatamente 1 (ou 100%)
    return portfolio

# Função de penalidade para grandes exposições
def penalidade_alocacao(portfolio, max_alocacao=0.2):
    penalidade = np.sum(np.maximum(portfolio - max_alocacao, 0))
    return penalidade

# Função de fitness com penalidade
def fitness_com_penalidade(portfolio, retornos, riscos, taxa_livre_risco, penalidade_peso=0.01):
    sharpe_ratio = calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco)
    penalidade = penalidade_alocacao(portfolio)
    return sharpe_ratio - penalidade_peso * penalidade

# Algoritmo Genético com Elitismo e penalidade
def algoritmo_genetico_com_elitismo(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100, elitismo=True):
    populacao = gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, len(retornos))
    melhor_portfolio = genoma_inicial
    melhor_sharpe = fitness_com_penalidade(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o fitness de cada portfólio
        fitness_scores = np.array([fitness_com_penalidade(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Elitismo: preservar o melhor indivíduo
        indice_melhor_portfolio = np.argmax(fitness_scores)
        melhor_da_geracao = populacao[indice_melhor_portfolio]
        melhor_fitness_da_geracao = fitness_scores[indice_melhor_portfolio]

        if melhor_fitness_da_geracao > melhor_sharpe:
            melhor_sharpe = melhor_fitness_da_geracao
            melhor_portfolio = melhor_da_geracao

        # Seleção, crossover e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []

        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i + 1]
            filho1, filho2 = cruzamento(pai1, pai2)
            filho1 = mutacao(filho1)
            filho2 = mutacao(filho2)

            # Garantir que a soma das alocações seja 100%
            filho1 = filho1 / filho1.sum()
            filho2 = filho2 / filho2.sum()

            nova_populacao.append(filho1)
            nova_populacao.append(filho2)

        # Inserir o melhor portfólio (elitismo)
        if elitismo:
            nova_populacao[0] = melhor_da_geracao

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Exemplo de dados reais para retornos e riscos
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com o genoma inicial fixo, elitismo e penalidade
melhor_portfolio = algoritmo_genetico_com_elitismo(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Função para salvar o DataFrame em um novo CSV
distribuicao_df.to_csv('/content/Pool_Investimentos_Atualizado.csv', index=False)

# Exibir a distribuição ideal do investimento
print(distribuicao_df)

# Exibir o melhor portfólio e seu retorno em 12, 24 e 36 meses
print(f"Melhor portfólio: {melhor_portfolio}")
retorno_12m = np.dot(melhor_portfolio, retornos_12m)
retorno_24m = np.dot(melhor_portfolio, retornos_24m)
retorno_36m = np.dot(melhor_portfolio, retornos_36m)
print(f"Retorno em 12 meses: {retorno_12m}")
print(f"Retorno em 24 meses: {retorno_24m}")
print(f"Retorno em 36 meses: {retorno_36m}")

[*********************100%***********************]  15 of 15 completed


Verificando valores NaN após preenchimento:
Ticker
ADA-USD     0
B3SA3.SA    0
BRL=X       0
BTC-USD     0
EMBR3.SA    0
ETH-USD     0
GOLL4.SA    0
ITUB4.SA    0
JBSS3.SA    0
LTC-USD     0
MGLU3.SA    0
PETR4.SA    0
RENT3.SA    0
VALE3.SA    0
WEGE3.SA    0
dtype: int64
Geracao 1, Melhor Sharpe Ratio: 0.15440362649978298
Geracao 2, Melhor Sharpe Ratio: 0.24118315036039226
Geracao 3, Melhor Sharpe Ratio: 0.40140791013062277
Geracao 4, Melhor Sharpe Ratio: 0.40140791013062277
Geracao 5, Melhor Sharpe Ratio: 0.4564324875447454
Geracao 6, Melhor Sharpe Ratio: 0.4745371845197569
Geracao 7, Melhor Sharpe Ratio: 0.601994815821446
Geracao 8, Melhor Sharpe Ratio: 0.6183599896891153
Geracao 9, Melhor Sharpe Ratio: 0.6378767076622528
Geracao 10, Melhor Sharpe Ratio: 0.6671714390607449
Geracao 11, Melhor Sharpe Ratio: 0.6813290012291248
Geracao 12, Melhor Sharpe Ratio: 0.7093916453870173
Geracao 13, Melhor Sharpe Ratio: 0.7232747685433303
Geracao 14, Melhor Sharpe Ratio: 0.7260794285135721
Gera

In [4]:
#Aplicando maior penalidade, normalização e elitismo
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado (deve conter 34 ativos)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN nos dados históricos com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Verificar se ainda existem valores NaN
print("Verificando valores NaN após preenchimento:")
print(dados_historicos_completos.isna().sum())

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função para garantir que não há alocações negativas ou acima de 20%
def ajustar_alocacao(portfolio):
    portfolio = np.clip(portfolio, 0, 0.2)  # Limitar entre 0 e 20%
    portfolio /= portfolio.sum()  # Normalizar para garantir que a soma seja 1
    return portfolio

# Função de mutação ajustada para evitar valores negativos e respeitar limite de 20%
def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)
        portfolio = ajustar_alocacao(portfolio)  # Garantir que os valores estejam entre 0 e 20% e normalizar
    return portfolio

# Função de cruzamento de ponto único ajustada
def cruzamento(pai1, pai2):
    ponto_corte = np.random.randint(1, len(pai1) - 1)
    filho1 = np.concatenate((pai1[:ponto_corte], pai2[ponto_corte:]))
    filho2 = np.concatenate((pai2[:ponto_corte:], pai1[ponto_corte:]))

    # Ajustar e normalizar os filhos
    filho1 = ajustar_alocacao(filho1)
    filho2 = ajustar_alocacao(filho2)

    return filho1, filho2

# Função para gerar o genoma inicial de portfólios com 34 ativos
genoma_inicial = np.array([
    0.00,  # Tesouro Prefixado (sem alocação)
    0.00,  # Tesouro RendA (sem alocação)
    0.20,  # Tesouro Selic (20% do portfólio)
    0.00,  # Tesouro IPCA (sem alocação)
    0.05,  # Bitcoin (5% do portfólio)
    0.00,  # Cardano (sem alocação)
    0.03,  # Ethereum (5% do portfólio)
    0.00,  # Litecoin (sem alocação)
    0.00,  # Dólar (sem alocação)
    0.03,  # VALE3.SA (5% do portfólio)
    0.05,  # PETR4.SA (5% do portfólio)
    0.00,  # JBSS3.SA (sem alocação)
    0.00,  # MGLU3.SA (sem alocação)
    0.00,  # RENT3.SA (sem alocação)
    0.00,  # B3SA3.SA (sem alocação)
    0.00,  # WEGE3.SA (sem alocação)
    0.00,  # EMBR3.SA (sem alocação)
    0.05,  # GOLL4.SA (5% do portfólio)
    0.05,  # ITUB4.SA (5% do portfólio)
    0.06,  # Renda Fixa BB 1 (10% do portfólio)
    0.10,  # Renda Fixa BB 2 (10% do portfólio)
    0.00,  # Renda Fixa BB 3 (sem alocação)
    0.00,  # Renda Fixa BB 4 (sem alocação)
    0.00,  # Renda Fixa BB 5 (sem alocação)
    0.05,  # Renda Fixa Bradesco 1 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 2 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 3 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 4 (5% do portfólio)
    0.00,  # Renda Fixa Bradesco 5 (sem alocação)
    0.05,  # Renda Fixa Itaú 1 (5% do portfólio)
    0.05,  # Renda Fixa Itaú 2 (5% do portfólio)
    0.03,  # Renda Fixa Itaú 3 (5% do portfólio)
    0.05,  # Renda Fixa Itaú 4 (5% do portfólio)
    0.00   # Renda Fixa Itaú 5 (sem alocação)
])

# Verificando se a soma das alocações é 100%
assert np.isclose(genoma_inicial.sum(), 1.0), "As alocações devem somar 100% (ou 1.0 em fração)"

# Função para gerar a população inicial com o genoma inicial fixo
def gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, num_ativos):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(num_ativos)))
    return populacao

# Função para rodar o algoritmo genético com genoma inicial fixo
def algoritmo_genetico_com_genoma_inicial(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100):
    populacao = gerar_portfolios_com_genoma_inicial(genoma_inicial, num_portfolios, len(retornos))
    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio (fitness) para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Verificar se algum portfólio é inválido (alocação fora do intervalo permitido)
        for portfolio in populacao:
            assert np.isclose(portfolio.sum(), 1.0), "Portfólio inválido: soma das alocações não é 100%"

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = cruzamento(pai1, pai2)

            # Garantir que os filhos estejam dentro dos limites
            filho1 = ajustar_alocacao(filho1)  # Limitar a alocação por ativo e normalizar
            filho2 = ajustar_alocacao(filho2)  # Limitar a alocação por ativo e normalizar

            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        # Inserir o elitismo: garantir que o melhor portfólio da geração anterior permaneça
        nova_populacao[0] = melhor_portfolio

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

# Exemplo de dados reais para retornos e riscos
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com o genoma inicial fixo
melhor_portfolio = algoritmo_genetico_com_genoma_inicial(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Função para salvar o DataFrame em um novo CSV
distribuicao_df.to_csv('/content/Pool_Investimentos_Atualizacao2.csv', index=False)

# Exibir a distribuição ideal do investimento
print(distribuicao_df)

# Calcular os retornos com base nas alocações
retorno_12m = np.dot(melhor_portfolio, retornos_12m)
retorno_24m = np.dot(melhor_portfolio, retornos_24m)
retorno_36m = np.dot(melhor_portfolio, retornos_36m)

print(f"Melhor portfólio: {melhor_portfolio}")
print(f"Retorno em 12 meses: {retorno_12m}")
print(f"Retorno em 24 meses: {retorno_24m}")
print(f"Retorno em 36 meses: {retorno_36m}")


[*********************100%***********************]  15 of 15 completed


Verificando valores NaN após preenchimento:
Ticker
ADA-USD     0
B3SA3.SA    0
BRL=X       0
BTC-USD     0
EMBR3.SA    0
ETH-USD     0
GOLL4.SA    0
ITUB4.SA    0
JBSS3.SA    0
LTC-USD     0
MGLU3.SA    0
PETR4.SA    0
RENT3.SA    0
VALE3.SA    0
WEGE3.SA    0
dtype: int64
Geracao 1, Melhor Sharpe Ratio: 0.27337926070971086
Geracao 2, Melhor Sharpe Ratio: 0.32333799657070217
Geracao 3, Melhor Sharpe Ratio: 0.3392222945912442
Geracao 4, Melhor Sharpe Ratio: 0.39327152856784986
Geracao 5, Melhor Sharpe Ratio: 0.4431441062233805
Geracao 6, Melhor Sharpe Ratio: 0.5430109672156958
Geracao 7, Melhor Sharpe Ratio: 0.5443195165665068
Geracao 8, Melhor Sharpe Ratio: 0.5632155909286117
Geracao 9, Melhor Sharpe Ratio: 0.5825647214335155
Geracao 10, Melhor Sharpe Ratio: 0.6073256400074445
Geracao 11, Melhor Sharpe Ratio: 0.6399478856358642
Geracao 12, Melhor Sharpe Ratio: 0.6555878039210521
Geracao 13, Melhor Sharpe Ratio: 0.6992710518712355
Geracao 14, Melhor Sharpe Ratio: 0.6994798347981092
Gera



---
# Testando novas funções de CrossOver

# Crossover de Dois Pontos

Vamos manter o mesmo genoma para comparar os métodos

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Baixar dados históricos para as ações, criptos e dólar (caso necessário)
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Função de crossover de dois pontos
def crossover_dois_pontos(pai1, pai2):
    # Garantir que os pontos de corte sejam diferentes e não nas extremidades
    ponto1 = np.random.randint(1, len(pai1) - 1)
    ponto2 = np.random.randint(ponto1 + 1, len(pai1))

    # Criar filhos trocando os genes entre os dois pontos de corte
    filho1 = np.concatenate((pai1[:ponto1], pai2[ponto1:ponto2], pai1[ponto2:]))
    filho2 = np.concatenate((pai2[:ponto1], pai1[ponto1:ponto2], pai2[ponto2:]))

    # Garantir que os filhos somem 1.0 (100% de alocação) após o crossover
    filho1 /= filho1.sum()
    filho2 /= filho2.sum()

    return filho1, filho2

# Genoma inicial
genoma_inicial = np.array([
    0.00,  # Tesouro Prefixado
    0.00,  # Tesouro RendA
    0.20,  # Tesouro Selic
    0.00,  # Tesouro IPCA
    0.05,  # Bitcoin
    0.00,  # Cardano
    0.03,  # Ethereum
    0.00,  # Litecoin
    0.00,  # Dólar
    0.03,  # VALE3.SA
    0.05,  # PETR4.SA
    0.00,  # JBSS3.SA
    0.00,  # MGLU3.SA
    0.00,  # RENT3.SA
    0.00,  # B3SA3.SA
    0.00,  # WEGE3.SA
    0.00,  # EMBR3.SA
    0.05,  # GOLL4.SA
    0.05,  # ITUB4.SA
    0.06,  # Renda Fixa BB 1
    0.10,  # Renda Fixa BB 2
    0.00,  # Renda Fixa BB 3
    0.00,  # Renda Fixa BB 4
    0.00,  # Renda Fixa BB 5
    0.05,  # Renda Fixa Bradesco 1
    0.05,  # Renda Fixa Bradesco 2
    0.05,  # Renda Fixa Bradesco 3
    0.05,  # Renda Fixa Bradesco 4
    0.00,  # Renda Fixa Bradesco 5
    0.05,  # Renda Fixa Itaú 1
    0.05,  # Renda Fixa Itaú 2
    0.03,  # Renda Fixa Itaú 3
    0.05,  # Renda Fixa Itaú 4
    0.00   # Renda Fixa Itaú 5
])

# Exemplo de execução do crossover de dois pontos
pai1 = genoma_inicial
pai2 = np.random.dirichlet(np.ones(len(genoma_inicial)))  # Gerar outro genoma aleatório para teste

filho1, filho2 = crossover_dois_pontos(pai1, pai2)

# Exibir os genomas resultantes
print("Genoma do Pai 1:", pai1)
print("Genoma do Pai 2:", pai2)
print("Filho 1 após crossover de dois pontos:", filho1)
print("Filho 2 após crossover de dois pontos:", filho2)


Genoma do Pai 1: [0.   0.   0.2  0.   0.05 0.   0.03 0.   0.   0.03 0.05 0.   0.   0.
 0.   0.   0.   0.05 0.05 0.06 0.1  0.   0.   0.   0.05 0.05 0.05 0.05
 0.   0.05 0.05 0.03 0.05 0.  ]
Genoma do Pai 2: [0.00105661 0.0334435  0.00245038 0.01410708 0.01412383 0.0465497
 0.12773237 0.02680751 0.01380842 0.01795133 0.03794788 0.04916865
 0.02184884 0.02058823 0.04771811 0.04226534 0.03888611 0.03456533
 0.02503923 0.06101914 0.06070737 0.01063721 0.01085551 0.00956763
 0.00406316 0.01785243 0.02781381 0.01843106 0.02544955 0.04277709
 0.00570221 0.0258089  0.02405363 0.03920285]
Filho 1 após crossover de dois pontos: [0.         0.         0.0026027  0.01498396 0.01500176 0.04944319
 0.1356721  0.02847384 0.01466674 0.01906716 0.04030669 0.
 0.         0.         0.         0.         0.         0.05310796
 0.05310796 0.06372955 0.10621592 0.         0.         0.
 0.05310796 0.05310796 0.05310796 0.05310796 0.         0.05310796
 0.05310796 0.03186477 0.05310796 0.        ]
Filho 2 ap

**Próximos Passos**

Agora, esses filhos passarão por avaliação com base no Sharpe Ratio e, eventualmente, os melhores serão selecionados para a próxima geração, enquanto a mutação pode introduzir ainda mais variabilidade.

In [ ]:
import numpy as np

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Taxa livre de risco (usando taxa Selic atual como exemplo)
taxa_livre_risco = 0.1075  # 10.75%

# Exemplo de retornos e riscos (valores simulados para cada ativo)
retornos_simulados = np.random.rand(34) * 0.2  # Simulando retornos entre 0 e 20%
riscos_simulados = np.random.rand(34) * 0.1    # Simulando riscos entre 0 e 10%

# Genoma do Filho 1 e Filho 2 após crossover de dois pontos
genoma_filho1 = np.array([0.00, 0.00, 0.20, 0.00, 0.05, 0.00, 0.03, 0.00, 0.00, 0.03,
                          0.05, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.05, 0.06,
                          0.10, 0.00, 0.00, 0.00, 0.05, 0.05, 0.05, 0.05, 0.00, 0.01174119,
                          0.00673219, 0.03, 0.05, 0.00])

genoma_filho2 = np.array([0.00587827, 0.05226215, 0.01566535, 0.04146163, 0.01390723, 0.03013254,
                          0.05556398, 0.02005116, 0.00932852, 0.00143567, 0.03362752, 0.04451197,
                          0.03117472, 0.00016837, 0.03875161, 0.02978987, 0.02565631, 0.03232319,
                          0.00112141, 0.01608215, 0.03452704, 0.01184425, 0.06840172, 0.05350863,
                          0.00264887, 0.11465835, 0.04597692, 0.05127688, 0.04814377, 0.05,
                          0.05, 0.01019189, 0.01056653, 0.03088817])

# Calcular o Sharpe Ratio para os filhos
sharpe_filho1 = calcular_sharpe(genoma_filho1, retornos_simulados, riscos_simulados, taxa_livre_risco)
sharpe_filho2 = calcular_sharpe(genoma_filho2, retornos_simulados, riscos_simulados, taxa_livre_risco)

print(f"Sharpe Ratio Filho 1: {sharpe_filho1}")
print(f"Sharpe Ratio Filho 2: {sharpe_filho2}")


Sharpe Ratio Filho 1: -0.3174590767925491
Sharpe Ratio Filho 2: -0.048282709933212035


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado (deve conter 34 ativos)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN nos dados históricos com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Usar a média de retornos de 12, 24 e 36 meses como retornos finais
retornos_completos = np.mean([retornos_12m, retornos_24m, retornos_36m], axis=0)

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função de crossover de dois pontos
def crossover_dois_pontos(pai1, pai2):
    ponto1 = np.random.randint(1, len(pai1) - 1)
    ponto2 = np.random.randint(ponto1 + 1, len(pai1))

    filho1 = np.concatenate((pai1[:ponto1], pai2[ponto1:ponto2], pai1[ponto2:]))
    filho2 = np.concatenate((pai2[:ponto1], pai1[ponto1:ponto2], pai2[ponto2:]))

    filho1 /= filho1.sum()  # Normalizar para somar 1
    filho2 /= filho2.sum()

    return filho1, filho2

# Função de mutação
def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas
        portfolio /= portfolio.sum()  # Normalizar
    return portfolio

# Função para rodar o algoritmo genético com crossover de dois pontos
def algoritmo_genetico_crossover_dois_pontos(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100):
    populacao = [genoma_inicial] + [np.random.dirichlet(np.ones(len(genoma_inicial))) for _ in range(num_portfolios - 1)]
    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = crossover_dois_pontos(pai1, pai2)
            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Função para seleção por torneio
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

# Definir o genoma inicial
genoma_inicial = np.array([
    0.00,  # Tesouro Prefixado (sem alocação)
    0.00,  # Tesouro RendA (sem alocação)
    0.20,  # Tesouro Selic (20% do portfólio)
    0.00,  # Tesouro IPCA (sem alocação)
    0.05,  # Bitcoin (5% do portfólio)
    0.00,  # Cardano (sem alocação)
    0.03,  # Ethereum (5% do portfólio)
    0.00,  # Litecoin (sem alocação)
    0.00,  # Dólar (sem alocação)
    0.03,  # VALE3.SA (5% do portfólio)
    0.05,  # PETR4.SA (5% do portfólio)
    0.00,  # JBSS3.SA (sem alocação)
    0.00,  # MGLU3.SA (sem alocação)
    0.00,  # RENT3.SA (sem alocação)
    0.00,  # B3SA3.SA (sem alocação)
    0.00,  # WEGE3.SA (sem alocação)
    0.00,  # EMBR3.SA (sem alocação)
    0.05,  # GOLL4.SA (5% do portfólio)
    0.05,  # ITUB4.SA (5% do portfólio)
    0.06,  # Renda Fixa BB 1 (10% do portfólio)
    0.10,  # Renda Fixa BB 2 (10% do portfólio)
    0.00,  # Renda Fixa BB 3 (sem alocação)
    0.00,  # Renda Fixa BB 4 (sem alocação)
    0.00,  # Renda Fixa BB 5 (sem alocação)
    0.05,  # Renda Fixa Bradesco 1 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 2 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 3 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 4 (5% do portfólio)
    0.00,  # Renda Fixa Bradesco 5 (sem alocação)
    0.05,  # Renda Fixa Itaú 1 (5% do portfólio)
    0.05,  # Renda Fixa Itaú 2 (5% do portfólio)
    0.03,  # Renda Fixa Itaú 3 (3% do portfólio)
    0.05,  # Renda Fixa Itaú 4 (5% do portfólio)
    0.00   # Renda Fixa Itaú 5 (sem alocação)
])

# Rodar o algoritmo genético com crossover de dois pontos
melhor_portfolio = algoritmo_genetico_crossover_dois_pontos(retornos_completos, riscos_completos, genoma_inicial)

# Exibir o melhor portfólio
print(f"Melhor Portfólio: {melhor_portfolio}")

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Exibir a distribuição ideal do investimento
print(distribuicao_df)



[*********************100%***********************]  15 of 15 completed


Geracao 1, Melhor Sharpe Ratio: 19.080129542537012
Geracao 2, Melhor Sharpe Ratio: 19.454736494011556
Geracao 3, Melhor Sharpe Ratio: 19.498235924021067
Geracao 4, Melhor Sharpe Ratio: 23.227896241775515
Geracao 5, Melhor Sharpe Ratio: 23.227896241775515
Geracao 6, Melhor Sharpe Ratio: 23.227896241775515
Geracao 7, Melhor Sharpe Ratio: 26.999249909373653
Geracao 8, Melhor Sharpe Ratio: 27.620817192507875
Geracao 9, Melhor Sharpe Ratio: 28.012242517390938
Geracao 10, Melhor Sharpe Ratio: 28.426705627828497
Geracao 11, Melhor Sharpe Ratio: 28.463498190660413
Geracao 12, Melhor Sharpe Ratio: 29.646346400475718
Geracao 13, Melhor Sharpe Ratio: 30.516699979495282
Geracao 14, Melhor Sharpe Ratio: 30.516699979495282
Geracao 15, Melhor Sharpe Ratio: 30.803429752978218
Geracao 16, Melhor Sharpe Ratio: 31.40313154627469
Geracao 17, Melhor Sharpe Ratio: 31.703182677254503
Geracao 18, Melhor Sharpe Ratio: 31.746313939391147
Geracao 19, Melhor Sharpe Ratio: 32.61597399988491
Geracao 20, Melhor Shar

Para esse Sharpe Ratio tão alto vamos tentar duas soluções nesse CROSSOVER DE DOIS PONTOS:
1. Normalização de Dados:
Vamos normalizar os retornos e os riscos para garantir que os valores estejam na mesma escala. Isso ajudará a evitar que ativos com retornos ou riscos extremos distorçam o Sharpe Ratio. (caso das criptos)
A normalização pode ser feita usando o z-score ou escalando os valores entre 0 e 1. Para o nosso caso, usar o z-score parece mais apropriado, pois mantém a distribuição dos dados.
2. Penalização para Grandes Exposições:
Vamos aplicar uma penalização caso o portfólio esteja excessivamente concentrado em poucos ativos.
Podemos fazer isso incluindo uma função de penalidade que aumente à medida que a alocação em um ativo ultrapassa uma certa porcentagem do portfólio, ou quando a diversidade do portfólio é baixa.
3. Penalização do fitness
4. Introdução do elitismo
5. Aplicação da penalização e restrições aos filhos , crossover e mutação


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Carregar os dados do CSV atualizado (substitua o caminho do arquivo conforme necessário)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos para renda fixa e tesouro, arredondados para 2 casas decimais
ativos_fixa_tesouro = df[df['Ativo'].str.contains('Tesouro|Renda Fixa')]
retornos_fixa_tesouro = np.round(ativos_fixa_tesouro[['Rentabilidade 12 meses', 'Rentabilidade 24 meses', 'Rentabilidade 36 meses']].mean(axis=1).values, 2)

# Extrair riscos para renda fixa e tesouro, arredondados para 2 casas decimais
riscos_fixa_tesouro = np.round(ativos_fixa_tesouro['Taxa de Risco'].values, 2)

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptomoedas
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptomoedas e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Normalizar os retornos e riscos usando z-score
scaler = StandardScaler()
retornos_normalizados = scaler.fit_transform(retornos_diarios_completos.mean().values.reshape(-1, 1)).flatten()
riscos_normalizados = scaler.fit_transform(riscos_acoes_cripto_dolar.values.reshape(-1, 1)).flatten()

# Combinar os retornos e riscos de ações/cripto com os de renda fixa e tesouro
retornos_completos = np.concatenate((retornos_normalizados, retornos_fixa_tesouro))
riscos_completos = np.concatenate((riscos_normalizados, riscos_fixa_tesouro))

# Função de penalização para portfólios excessivamente concentrados
def penalizacao(portfolio, max_alocacao=0.2):
    penalidade = np.sum(np.maximum(0, portfolio - max_alocacao) ** 2)  # Penalidade quadrática
    return penalidade

# Função para calcular o Sharpe Ratio com penalização para grandes exposições
def calcular_sharpe_penalizado(portfolio, retornos, riscos, taxa_livre_risco, penalizacao=0.1):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio

    # Penalizar alocações grandes
    penalidade = penalizacao * np.sum(portfolio ** 2)  # Penalização para grandes alocações
    return sharpe_ratio - penalidade  # Reduzir o Sharpe Ratio com base na penalidade

# Função de crossover de dois pontos
def cruzamento_dois_pontos(pai1, pai2):
    ponto_corte1 = np.random.randint(1, len(pai1) - 1)
    ponto_corte2 = np.random.randint(ponto_corte1, len(pai1) - 1)
    filho1 = np.concatenate((pai1[:ponto_corte1], pai2[ponto_corte1:ponto_corte2], pai1[ponto_corte2:]))
    filho2 = np.concatenate((pai2[:ponto_corte1], pai1[ponto_corte1:ponto_corte2], pai2[ponto_corte2:]))
    return filho1, filho2

# Função de mutação com restrição de 20%
def mutacao(portfolio, taxa_mutacao=0.01, max_alocacao=0.2):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.05, 0.05)
        portfolio = np.clip(portfolio, 0, max_alocacao)  # Limitar alocações entre 0 e 20%
        portfolio /= portfolio.sum()  # Normalizar para garantir que a soma das alocações seja 1
    return portfolio

# Função de seleção por torneio
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

# Algoritmo genético com penalização e elitismo
def algoritmo_genetico_penalizado(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100, penalizacao=0.1, elitismo=True):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(len(genoma_inicial))))

    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe_penalizado(genoma_inicial, retornos, riscos, taxa_livre_risco, penalizacao)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio com penalização para cada portfólio
        fitness_scores = np.array([calcular_sharpe_penalizado(port, retornos, riscos, taxa_livre_risco, penalizacao) for port in populacao])

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = cruzamento_dois_pontos(pai1, pai2)

            # Aplicar mutação e garantir restrição de alocação
            filho1 = mutacao(filho1)
            filho2 = mutacao(filho2)

            nova_populacao.append(filho1)
            nova_populacao.append(filho2)

        # Aplicar elitismo, mantendo o melhor portfólio da geração anterior
        if elitismo:
            nova_populacao[0] = melhor_portfolio

        populacao = nova_populacao

        # Exibir o Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio com penalidade: {melhor_sharpe}")

    return melhor_portfolio

# Rodar o algoritmo genético com penalização e elitismo
melhor_portfolio_penalizado = algoritmo_genetico_penalizado(retornos_completos, riscos_completos, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento_penalizado = melhor_portfolio_penalizado * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df_penalizado = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio_penalizado * 100,
    'Valor Investido (R$)': distribuicao_investimento_penalizado
})

# Exibir a distribuição ideal do investimento com arredondamento
distribuicao_df_penalizado['Alocacao (%)'] = distribuicao_df_penalizado['Alocacao (%)'].round(2)
distribuicao_df_penalizado['Valor Investido (R$)'] = distribuicao_df_penalizado['Valor Investido (R$)'].round(2)

print(distribuicao_df_penalizado)

# Função para salvar o DataFrame em um novo CSV
distribuicao_df_penalizado.to_csv('/content/Distribuicao_Investimento_Penalizado.csv', index=False)

# Exibir o melhor portfólio e os retornos esperados arredondados para 2 casas decimais
# Certifique-se de usar os retornos completos dos 34 ativos
retorno_12m = round(np.dot(melhor_portfolio_penalizado, retornos_completos[:34]), 2)  # Retorno em 12 meses
retorno_24m = round(np.dot(melhor_portfolio_penalizado, retornos_completos[:34]), 2)  # Retorno em 24 meses
retorno_36m = round(np.dot(melhor_portfolio_penalizado, retornos_completos[:34]), 2)  # Retorno em 36 meses

melhor_portfolio_arredondado = np.round(melhor_portfolio_penalizado, 2)  # Arredondar melhor portfólio

print(f"Melhor portfólio: {melhor_portfolio_arredondado}")
print(f"Retorno em 12 meses: {retorno_12m}")
print(f"Retorno em 24 meses: {retorno_24m}")
print(f"Retorno em 36 meses: {retorno_36m}")






[*********************100%***********************]  15 of 15 completed


Geracao 1, Melhor Sharpe Ratio com penalidade: 20.055667703250776
Geracao 2, Melhor Sharpe Ratio com penalidade: 20.055667703250776
Geracao 3, Melhor Sharpe Ratio com penalidade: 21.464599425730032
Geracao 4, Melhor Sharpe Ratio com penalidade: 25.87657391786569
Geracao 5, Melhor Sharpe Ratio com penalidade: 26.594970309246737
Geracao 6, Melhor Sharpe Ratio com penalidade: 29.63950674148279
Geracao 7, Melhor Sharpe Ratio com penalidade: 31.403465764029658
Geracao 8, Melhor Sharpe Ratio com penalidade: 31.654866299069383
Geracao 9, Melhor Sharpe Ratio com penalidade: 34.33903942554506
Geracao 10, Melhor Sharpe Ratio com penalidade: 35.333966130277496
Geracao 11, Melhor Sharpe Ratio com penalidade: 36.064805358106405
Geracao 12, Melhor Sharpe Ratio com penalidade: 37.05849534206021
Geracao 13, Melhor Sharpe Ratio com penalidade: 40.28980796493459
Geracao 14, Melhor Sharpe Ratio com penalidade: 41.08647007351008
Geracao 15, Melhor Sharpe Ratio com penalidade: 42.9105061188412
Geracao 16, 

In [ ]:
#Ajustando a penalidade do fitness
# Ajustar a penalização para uma aplicação mais moderada
def calcular_sharpe_penalizado(portfolio, retornos, riscos, taxa_livre_risco, penalizacao=0.05):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado

    # Garantir que o risco não seja zero para evitar divisão por zero
    if risco_portfolio == 0:
        return -np.inf  # Penalizar fortemente se o risco for zero

    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio

    # Penalizar alocações maiores que 15%
    penalidade = penalizacao * np.sum(portfolio[portfolio > 0.15] ** 2)

    # Retornar o Sharpe Ratio penalizado
    return sharpe_ratio - penalidade

# Ajustar a chamada do algoritmo genético com a penalização ajustada
melhor_portfolio_penalizado = algoritmo_genetico_penalizado(retornos_completos, riscos_completos, genoma_inicial, penalizacao=0.05)

# Exibir os resultados, incluindo o Sharpe Ratio corrigido
melhor_portfolio_arredondado = np.round(melhor_portfolio_penalizado, 2)
print(f"Melhor portfólio: {melhor_portfolio_arredondado}")

# Calcular os retornos com base no portfólio ajustado
retorno_12m = round(np.dot(melhor_portfolio_penalizado, retornos_completos), 2)
retorno_24m = round(np.dot(melhor_portfolio_penalizado, retornos_completos), 2)
retorno_36m = round(np.dot(melhor_portfolio_penalizado, retornos_completos), 2)

print(f"Retorno em 12 meses: {retorno_12m}")
print(f"Retorno em 24 meses: {retorno_24m}")
print(f"Retorno em 36 meses: {retorno_36m}")



Geracao 1, Melhor Sharpe Ratio com penalidade: 19.178359126014477
Geracao 2, Melhor Sharpe Ratio com penalidade: 22.777033282069354
Geracao 3, Melhor Sharpe Ratio com penalidade: 26.437941667391847
Geracao 4, Melhor Sharpe Ratio com penalidade: 29.730344278310728
Geracao 5, Melhor Sharpe Ratio com penalidade: 29.774802848842825
Geracao 6, Melhor Sharpe Ratio com penalidade: 32.23860818629813
Geracao 7, Melhor Sharpe Ratio com penalidade: 37.77061591858993
Geracao 8, Melhor Sharpe Ratio com penalidade: 37.908017354251776
Geracao 9, Melhor Sharpe Ratio com penalidade: 42.07141813697689
Geracao 10, Melhor Sharpe Ratio com penalidade: 44.051582615925774
Geracao 11, Melhor Sharpe Ratio com penalidade: 44.07063685649913
Geracao 12, Melhor Sharpe Ratio com penalidade: 46.13495851211772
Geracao 13, Melhor Sharpe Ratio com penalidade: 47.204424405135384
Geracao 14, Melhor Sharpe Ratio com penalidade: 47.204424405135384
Geracao 15, Melhor Sharpe Ratio com penalidade: 48.339656289812524
Geracao 1

In [ ]:
# Função de penalização ajustada com verificação da taxa Selic
def calcular_sharpe_penalizado(portfolio, retornos, riscos, taxa_livre_risco, penalizacao=0.2):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado

    # Verificar se o risco é muito pequeno
    if risco_portfolio < 0.01:  # Definir um limite mínimo para o risco
        return -np.inf  # Penalizar fortemente se o risco for muito baixo (quase zero)

    # Penalizar fortemente se o retorno for menor que a taxa Selic
    if retorno_portfolio < taxa_livre_risco:
        return -np.inf  # Penalizar fortemente se o retorno for menor que a taxa Selic

    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio

    # Penalizar alocações maiores que 15%
    penalidade = penalizacao * np.sum((portfolio[portfolio > 0.15] - 0.15) ** 2)

    # Retornar o Sharpe Ratio penalizado
    return sharpe_ratio - penalidade

# Reescalar retornos e riscos para garantir que ambos estejam na mesma magnitude
def reescalar_dados(retornos, riscos):
    max_retorno = np.max(retornos)
    max_risco = np.max(riscos)

    # Normalizar retornos e riscos
    retornos_normalizados = retornos / max_retorno
    riscos_normalizados = riscos / max_risco

    return retornos_normalizados, riscos_normalizados

# Ajustar a chamada do algoritmo genético
retornos_normalizados, riscos_normalizados = reescalar_dados(retornos_completos, riscos_completos)

# Rodar o algoritmo genético com penalização
melhor_portfolio_penalizado = algoritmo_genetico_penalizado(retornos_normalizados, riscos_normalizados, genoma_inicial, penalizacao=0.2)

# Exibir os resultados, incluindo o Sharpe Ratio corrigido
melhor_portfolio_arredondado = np.round(melhor_portfolio_penalizado, 2)
print(f"Melhor portfólio: {melhor_portfolio_arredondado}")

# Calcular os retornos com base no portfólio ajustado
retorno_12m = round(np.dot(melhor_portfolio_penalizado, retornos_normalizados), 2)
retorno_24m = round(np.dot(melhor_portfolio_penalizado, retornos_normalizados), 2)
retorno_36m = round(np.dot(melhor_portfolio_penalizado, retornos_normalizados), 2)

# Garantir que o retorno não seja menor que a taxa Selic
if retorno_12m < taxa_livre_risco:
    retorno_12m = taxa_livre_risco
if retorno_24m < taxa_livre_risco:
    retorno_24m = taxa_livre_risco
if retorno_36m < taxa_livre_risco:
    retorno_36m = taxa_livre_risco

print(f"Retorno em 12 meses: {retorno_12m}")
print(f"Retorno em 24 meses: {retorno_24m}")
print(f"Retorno em 36 meses: {retorno_36m}")


Geracao 1, Melhor Sharpe Ratio com penalidade: 2.4896015398100326
Geracao 2, Melhor Sharpe Ratio com penalidade: 2.6696807821673123
Geracao 3, Melhor Sharpe Ratio com penalidade: 3.043665769091461
Geracao 4, Melhor Sharpe Ratio com penalidade: 3.3777209225830482
Geracao 5, Melhor Sharpe Ratio com penalidade: 3.782386976611323
Geracao 6, Melhor Sharpe Ratio com penalidade: 4.277093384605259
Geracao 7, Melhor Sharpe Ratio com penalidade: 4.591938097748261
Geracao 8, Melhor Sharpe Ratio com penalidade: 4.987318657627813
Geracao 9, Melhor Sharpe Ratio com penalidade: 5.196722312519023
Geracao 10, Melhor Sharpe Ratio com penalidade: 5.3868913593812335
Geracao 11, Melhor Sharpe Ratio com penalidade: 5.809301591006229
Geracao 12, Melhor Sharpe Ratio com penalidade: 5.887828430710245
Geracao 13, Melhor Sharpe Ratio com penalidade: 6.175745320953358
Geracao 14, Melhor Sharpe Ratio com penalidade: 6.32220244386185
Geracao 15, Melhor Sharpe Ratio com penalidade: 6.6223343211271795
Geracao 16, Mel

# Crossover de Múltiplos Pontos:

dividir os genes (alocações) dos pais em várias seções e realizar o cruzamento em vários pontos. Isso tende a gerar combinações mais diversificadas e pode ajudar a encontrar uma solução melhor.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado (deve conter 34 ativos)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN nos dados históricos com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Genoma inicial fixo
genoma_inicial = np.array([
    0.00,  # Tesouro Prefixado
    0.00,  # Tesouro RendA
    0.20,  # Tesouro Selic
    0.00,  # Tesouro IPCA
    0.05,  # Bitcoin
    0.00,  # Cardano
    0.03,  # Ethereum
    0.00,  # Litecoin
    0.00,  # Dólar
    0.03,  # VALE3.SA
    0.05,  # PETR4.SA
    0.00,  # JBSS3.SA
    0.00,  # MGLU3.SA
    0.00,  # RENT3.SA
    0.00,  # B3SA3.SA
    0.00,  # WEGE3.SA
    0.00,  # EMBR3.SA
    0.05,  # GOLL4.SA
    0.05,  # ITUB4.SA
    0.06,  # Renda Fixa BB 1
    0.10,  # Renda Fixa BB 2
    0.00,  # Renda Fixa BB 3
    0.00,  # Renda Fixa BB 4
    0.00,  # Renda Fixa BB 5
    0.05,  # Renda Fixa Bradesco 1
    0.05,  # Renda Fixa Bradesco 2
    0.05,  # Renda Fixa Bradesco 3
    0.05,  # Renda Fixa Bradesco 4
    0.00,  # Renda Fixa Bradesco 5
    0.05,  # Renda Fixa Itaú 1
    0.05,  # Renda Fixa Itaú 2
    0.03,  # Renda Fixa Itaú 3
    0.05,  # Renda Fixa Itaú 4
    0.00   # Renda Fixa Itaú 5
])

# Função de crossover de múltiplos pontos
def crossover_multiponto(pai1, pai2, num_pontos=3):
    pontos_corte = sorted(np.random.choice(range(1, len(pai1)), num_pontos, replace=False))
    filho1, filho2 = pai1.copy(), pai2.copy()
    for i in range(0, len(pontos_corte), 2):
        filho1[pontos_corte[i]:pontos_corte[i+1]] = pai2[pontos_corte[i]:pontos_corte[i+1]]
        filho2[pontos_corte[i]:pontos_corte[i+1]] = pai1[pontos_corte[i]:pontos_corte[i+1]]
    return filho1, filho2

# Função para rodar o algoritmo genético com crossover de múltiplos pontos
def algoritmo_genetico_com_crossover_multiponto(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(len(genoma_inicial))))

    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = crossover_multiponto(pai1, pai2)
            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

# Função de mutação
def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas (entre 0 e 1)
        portfolio /= portfolio.sum()  # Normalizar para garantir que a soma das alocações seja 1
    return portfolio

# Função para rodar o algoritmo genético com crossover de múltiplos pontos
def algoritmo_genetico_com_crossover_multiponto(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(len(genoma_inicial))))

    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = crossover_multiponto(pai1, pai2)
            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

# Função de crossover de múltiplos pontos corrigida
def crossover_multiponto(pai1, pai2, num_pontos=3):
    # Garantir que o número de pontos de corte seja par
    if num_pontos % 2 != 0:
        num_pontos += 1

    # Gerar pontos de corte e ordená-los
    pontos_corte = sorted(np.random.choice(range(1, len(pai1)), num_pontos, replace=False))

    # Criar cópias dos pais para os filhos
    filho1, filho2 = pai1.copy(), pai2.copy()

    # Aplicar crossover entre os pontos de corte
    for i in range(0, len(pontos_corte), 2):
        filho1[pontos_corte[i]:pontos_corte[i+1]] = pai2[pontos_corte[i]:pontos_corte[i+1]]
        filho2[pontos_corte[i]:pontos_corte[i+1]] = pai1[pontos_corte[i]:pontos_corte[i+1]]

    return filho1, filho2

# Exemplo de dados reais para retornos e riscos (substitua pelos seus dados)
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com o genoma inicial fixo e crossover de múltiplos pontos
melhor_portfolio = algoritmo_genetico_com_crossover_multiponto(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Exibir a distribuição ideal do investimento
print(distribuicao_df)


[*********************100%***********************]  15 of 15 completed


Geracao 1, Melhor Sharpe Ratio: 0.17151436121326516
Geracao 2, Melhor Sharpe Ratio: 0.4808234874533664
Geracao 3, Melhor Sharpe Ratio: 0.5970090989051544
Geracao 4, Melhor Sharpe Ratio: 0.8355191653878286
Geracao 5, Melhor Sharpe Ratio: 0.9827530443598146
Geracao 6, Melhor Sharpe Ratio: 1.2707110312354726
Geracao 7, Melhor Sharpe Ratio: 1.3607792957578417
Geracao 8, Melhor Sharpe Ratio: 1.5382066540895365
Geracao 9, Melhor Sharpe Ratio: 1.5774517610435155
Geracao 10, Melhor Sharpe Ratio: 1.7050983667396142
Geracao 11, Melhor Sharpe Ratio: 1.7183190294531347
Geracao 12, Melhor Sharpe Ratio: 1.8029417228106501
Geracao 13, Melhor Sharpe Ratio: 1.8927178832086458
Geracao 14, Melhor Sharpe Ratio: 1.9696926509669763
Geracao 15, Melhor Sharpe Ratio: 2.0711545202446557
Geracao 16, Melhor Sharpe Ratio: 2.093982257546837
Geracao 17, Melhor Sharpe Ratio: 2.1099720722853834
Geracao 18, Melhor Sharpe Ratio: 2.1550090078744613
Geracao 19, Melhor Sharpe Ratio: 2.2167652390741646
Geracao 20, Melhor Sh

# Aplicando as validações no crossover de múltiplos pontos

1. Controle da população para que não seja superior a 100% após crossover e mutação
2. Normalização
3. Penalização no fitness
4. Elitismo


In [23]:
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado (deve conter 34 ativos)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptomoedas
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN nos dados históricos com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptomoedas e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função de penalização para portfólios excessivamente concentrados
def penalizacao(portfolio, max_alocacao=0.15):
    penalidade = 0
    for alocacao in portfolio:
        if alocacao > max_alocacao:
            penalidade += (alocacao - max_alocacao) ** 2  # Penalidade quadrática
    return penalidade

# Função de crossover de múltiplos pontos
def crossover_multiponto(pai1, pai2, num_pontos=3):
    # Garantir que o número de pontos de corte seja par
    if num_pontos % 2 != 0:
        num_pontos += 1

    # Gerar pontos de corte e ordená-los
    pontos_corte = sorted(np.random.choice(range(1, len(pai1)), num_pontos, replace=False))

    # Criar cópias dos pais para os filhos
    filho1, filho2 = pai1.copy(), pai2.copy()

    # Aplicar crossover entre os pontos de corte
    for i in range(0, len(pontos_corte), 2):
        filho1[pontos_corte[i]:pontos_corte[i+1]] = pai2[pontos_corte[i]:pontos_corte[i+1]]
        filho2[pontos_corte[i]:pontos_corte[i+1]] = pai1[pontos_corte[i]:pontos_corte[i+1]]

    return filho1, filho2

# Função de mutação com normalização para garantir que a soma das alocações seja 100%
def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas (entre 0 e 1)
        portfolio /= portfolio.sum()  # Normalizar para garantir que a soma das alocações seja 1
    return portfolio

# Função para rodar o algoritmo genético com crossover de múltiplos pontos e elitismo
def algoritmo_genetico_com_crossover_multiponto(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100, elitismo=0.1):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(len(genoma_inicial))))

    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) - penalizacao(port) for port in populacao])

        # Elitismo: manter os melhores portfólios
        num_elite = int(num_portfolios * elitismo)
        elite_indices = np.argsort(fitness_scores)[-num_elite:]
        elite = [populacao[i] for i in elite_indices]

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = crossover_multiponto(pai1, pai2)
            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        # Adicionar a elite de volta à população
        populacao = nova_populacao[:len(nova_populacao) - num_elite] + elite

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe:.2f}")

    return melhor_portfolio

# Funções auxiliares: seleção por torneio, cruzamento e mutação
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

# Exemplo de dados reais para retornos e riscos (substitua pelos seus dados)
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com o genoma inicial fixo e crossover de múltiplos pontos
melhor_portfolio = algoritmo_genetico_com_crossover_multiponto(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': np.round(melhor_portfolio * 100, 2),  # Arredondar para 2 casas decimais
    'Valor Investido (R$)': np.round(distribuicao_investimento, 2)  # Arredondar para 2 casas decimais
})

# Calcular o retorno em 12, 24 e 36 meses
retorno_12m = np.dot(melhor_portfolio, retornos_12m)
retorno_24m = np.dot(melhor_portfolio, retornos_24m)
retorno_36m = np.dot(melhor_portfolio, retornos_36m)

# Exibir o melhor portfólio, a distribuição e os retornos esperados
print("Melhor portfólio:")
print(np.round(melhor_portfolio, 2))
print("\nDistribuição ideal do investimento (R$):")
print(distribuicao_df)

# Exibir os retornos esperados
print(f"\nRetorno esperado em 12 meses: {retorno_12m:.2f}%")
print(f"Retorno esperado em 24 meses: {retorno_24m:.2f}%")
print(f"Retorno esperado em 36 meses: {retorno_36m:.2f}%")



[*********************100%***********************]  15 of 15 completed


Geracao 1, Melhor Sharpe Ratio: 0.60
Geracao 2, Melhor Sharpe Ratio: 0.90
Geracao 3, Melhor Sharpe Ratio: 1.08
Geracao 4, Melhor Sharpe Ratio: 1.43
Geracao 5, Melhor Sharpe Ratio: 1.59
Geracao 6, Melhor Sharpe Ratio: 1.80
Geracao 7, Melhor Sharpe Ratio: 2.07
Geracao 8, Melhor Sharpe Ratio: 2.31
Geracao 9, Melhor Sharpe Ratio: 2.40
Geracao 10, Melhor Sharpe Ratio: 2.43
Geracao 11, Melhor Sharpe Ratio: 2.52
Geracao 12, Melhor Sharpe Ratio: 2.69
Geracao 13, Melhor Sharpe Ratio: 2.79
Geracao 14, Melhor Sharpe Ratio: 2.84
Geracao 15, Melhor Sharpe Ratio: 2.98
Geracao 16, Melhor Sharpe Ratio: 3.02
Geracao 17, Melhor Sharpe Ratio: 3.05
Geracao 18, Melhor Sharpe Ratio: 3.20
Geracao 19, Melhor Sharpe Ratio: 3.23
Geracao 20, Melhor Sharpe Ratio: 3.25
Geracao 21, Melhor Sharpe Ratio: 3.31
Geracao 22, Melhor Sharpe Ratio: 3.35
Geracao 23, Melhor Sharpe Ratio: 3.35
Geracao 24, Melhor Sharpe Ratio: 3.37
Geracao 25, Melhor Sharpe Ratio: 3.37
Geracao 26, Melhor Sharpe Ratio: 3.37
Geracao 27, Melhor Sh

# Implementação do Blend Crossover (BLX-α)

Vamos considerar α = 0.5 para este exemplo, o que significa que os novos filhos poderão ter valores entre uma distância de 50% além dos valores dos pais.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

# Função para realizar o BLX-alpha crossover
def blend_crossover(pai1, pai2, alpha=0.5):
    filho1 = np.zeros_like(pai1)
    filho2 = np.zeros_like(pai2)

    for i in range(len(pai1)):
        # Encontrar o intervalo mínimo e máximo
        menor = min(pai1[i], pai2[i])
        maior = max(pai1[i], pai2[i])

        # Calcular a extensão do intervalo
        d = maior - menor

        # Gerar filhos no intervalo expandido pelo alpha
        filho1[i] = np.random.uniform(menor - alpha * d, maior + alpha * d)
        filho2[i] = np.random.uniform(menor - alpha * d, maior + alpha * d)

    # Garantir que os filhos respeitem o intervalo [0, 1]
    filho1 = np.clip(filho1, 0, 1)
    filho2 = np.clip(filho2, 0, 1)

    return filho1, filho2

# Função para rodar o algoritmo genético com BLX-alpha crossover
def algoritmo_genetico_com_blx_alpha(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100, alpha=0.5):
    populacao = [genoma_inicial]  # Começar com o genoma inicial fixo
    for _ in range(num_portfolios - 1):  # Gerar o restante aleatoriamente
        populacao.append(np.random.dirichlet(np.ones(len(genoma_inicial))))

    melhor_portfolio = genoma_inicial
    melhor_sharpe = calcular_sharpe(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o Sharpe Ratio para cada portfólio
        fitness_scores = np.array([calcular_sharpe(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_sharpe:
            melhor_sharpe = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento usando BLX-alpha
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]
            filho1, filho2 = blend_crossover(pai1, pai2, alpha)
            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_sharpe}")

    return melhor_portfolio

# Função de seleção por torneio e mutação já implementadas no código anterior
# Mutação já implementada no código anterior

# Rodar o algoritmo genético com o genoma inicial fixo e BLX-alpha crossover
melhor_portfolio_blx = algoritmo_genetico_com_blx_alpha(retornos_reais, riscos_reais, genoma_inicial, alpha=0.5)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio_blx * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio_blx * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Exibir a distribuição ideal do investimento
print(distribuicao_df)


Geracao 1, Melhor Sharpe Ratio: 0.07928909133602083
Geracao 2, Melhor Sharpe Ratio: 0.4925741301086632
Geracao 3, Melhor Sharpe Ratio: 0.7864667043294788
Geracao 4, Melhor Sharpe Ratio: 1.2494408448013747
Geracao 5, Melhor Sharpe Ratio: 1.4866483912155595
Geracao 6, Melhor Sharpe Ratio: 1.928724368613652
Geracao 7, Melhor Sharpe Ratio: 2.714857610553189
Geracao 8, Melhor Sharpe Ratio: 3.0046981268573165
Geracao 9, Melhor Sharpe Ratio: 3.3290340039244013
Geracao 10, Melhor Sharpe Ratio: 3.925528995094535
Geracao 11, Melhor Sharpe Ratio: 3.9272820941143176
Geracao 12, Melhor Sharpe Ratio: 4.9515475010693235
Geracao 13, Melhor Sharpe Ratio: 5.303573052284284
Geracao 14, Melhor Sharpe Ratio: 5.485362630287919
Geracao 15, Melhor Sharpe Ratio: 5.6982516337174305
Geracao 16, Melhor Sharpe Ratio: 6.104724101447083
Geracao 17, Melhor Sharpe Ratio: 6.641795005263044
Geracao 18, Melhor Sharpe Ratio: 6.667602725536025
Geracao 19, Melhor Sharpe Ratio: 7.093395272289622
Geracao 20, Melhor Sharpe Rat

In [ ]:
# Aqui vemos nosso sharpe ratio inconsistente
# Vamos tentar algumas soluções


Vamos implementar as seguintes melhorias no código:

Limitação de Alocação Máxima: Limitaremos a alocação de qualquer ativo para um máximo de 20% do portfólio.

Penalidade para Diversificação: Adicionaremos uma penalidade para incentivar a diversificação, evitando a concentração excessiva em poucos ativos.

Ajuste da Função de Fitness: Ajustaremos a função de fitness para considerar a diversidade no portfólio, além do Sharpe Ratio.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

# Carregar os dados do CSV atualizado (deve conter 34 ativos)
df = pd.read_csv('/content/Pool_Investimentos.csv')

# Extrair retornos do CSV para os 34 ativos
retornos_12m = df['Rentabilidade 12 meses'].values
retornos_24m = df['Rentabilidade 24 meses'].values
retornos_36m = df['Rentabilidade 36 meses'].values

# Lista de tickers das 15 ações, criptomoedas e dólar
tickers_acoes_cripto_dolar = ['VALE3.SA', 'PETR4.SA', 'JBSS3.SA', 'MGLU3.SA', 'RENT3.SA',
                              'B3SA3.SA', 'WEGE3.SA', 'EMBR3.SA', 'GOLL4.SA', 'ITUB4.SA',
                              'BTC-USD', 'ADA-USD', 'ETH-USD', 'LTC-USD', 'BRL=X']

# Baixar dados históricos de preços para as 15 ações e criptos
dados_historicos_completos = yf.download(tickers_acoes_cripto_dolar, start='2021-01-01', end='2024-01-01')['Adj Close']

# Preencher valores NaN nos dados históricos com a média da coluna correspondente
dados_historicos_completos.fillna(dados_historicos_completos.mean(), inplace=True)

# Calcular os retornos diários e o desvio padrão (volatilidade) anualizado para as 15 ações, criptos e dólar
retornos_diarios_completos = dados_historicos_completos.pct_change().dropna()
riscos_acoes_cripto_dolar = retornos_diarios_completos.std() * np.sqrt(252)  # Riscos anualizados (15 ativos)

# Definir riscos assumidos para os ativos de renda fixa e tesouro (totalizando 19 ativos)
riscos_fixa_tesouro = np.array([0.05, 0.06, 0.04, 0.03, 0.04, 0.05, 0.05, 0.05, 0.06, 0.04, 0.05, 0.03, 0.04, 0.06, 0.04, 0.05, 0.03, 0.04, 0.03])

# Combinar os riscos de ações, criptomoedas e renda fixa/tesouro para totalizar 34 ativos
riscos_completos_final = np.concatenate((riscos_acoes_cripto_dolar.values, riscos_fixa_tesouro))

# Função para calcular o Sharpe Ratio
def calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco):
    retorno_portfolio = np.dot(portfolio, retornos)  # Retorno ponderado
    risco_portfolio = np.sqrt(np.dot(portfolio, riscos ** 2))  # Risco ponderado
    sharpe_ratio = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return sharpe_ratio

# Função de penalidade para diversificação
def penalidade_diversificacao(portfolio):
    max_alocacao = 0.2  # Limitar a alocação máxima de qualquer ativo a 20%
    penalidade = np.sum(np.maximum(0, portfolio - max_alocacao))  # Penalizar qualquer alocação maior que 20%
    return penalidade

# Função de fitness com penalidade de diversificação
def fitness_com_penalidade(portfolio, retornos, riscos, taxa_livre_risco):
    sharpe = calcular_sharpe(portfolio, retornos, riscos, taxa_livre_risco)
    penalidade = penalidade_diversificacao(portfolio)
    return sharpe - penalidade  # O objetivo é maximizar o Sharpe Ratio e minimizar a penalidade

# Função para gerar o genoma inicial de portfólios com 34 ativos
genoma_inicial = np.array([
    0.00,  # Tesouro Prefixado (sem alocação)
    0.00,  # Tesouro RendA (sem alocação)
    0.20,  # Tesouro Selic (20% do portfólio)
    0.00,  # Tesouro IPCA (sem alocação)
    0.05,  # Bitcoin (5% do portfólio)
    0.00,  # Cardano (sem alocação)
    0.03,  # Ethereum (5% do portfólio)
    0.00,  # Litecoin (sem alocação)
    0.00,  # Dólar (sem alocação)
    0.03,  # VALE3.SA (5% do portfólio)
    0.05,  # PETR4.SA (5% do portfólio)
    0.00,  # JBSS3.SA (sem alocação)
    0.00,  # MGLU3.SA (sem alocação)
    0.00,  # RENT3.SA (sem alocação)
    0.00,  # B3SA3.SA (sem alocação)
    0.00,  # WEGE3.SA (sem alocação)
    0.00,  # EMBR3.SA (sem alocação)
    0.05,  # GOLL4.SA (5% do portfólio)
    0.05,  # ITUB4.SA (5% do portfólio)
    0.06,  # Renda Fixa BB 1 (10% do portfólio)
    0.10,  # Renda Fixa BB 2 (10% do portfólio)
    0.00,  # Renda Fixa BB 3 (sem alocação)
    0.00,  # Renda Fixa BB 4 (sem alocação)
    0.00,  # Renda Fixa BB 5 (sem alocação)
    0.05,  # Renda Fixa Bradesco 1 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 2 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 3 (5% do portfólio)
    0.05,  # Renda Fixa Bradesco 4 (5% do portfólio)
    0.00,  # Renda Fixa Bradesco 5 (sem alocação)
    0.05,  # Renda Fixa Itaú 1 (5% do portfólio)
    0.05,  # Renda Fixa Itaú 2 (5% do portfólio)
    0.03,  # Renda Fixa Itaú 3 (5% do portfólio)
    0.05,  # Renda Fixa Itaú 4 (5% do portfólio)
    0.00   # Renda Fixa Itaú 5 (sem alocação)
])

# Função para rodar o algoritmo genético com genoma inicial fixo e penalidade
def algoritmo_genetico_com_penalidade(retornos, riscos, genoma_inicial, taxa_livre_risco=0.1075, num_portfolios=100, geracoes=100):
    populacao = [genoma_inicial] + [np.random.dirichlet(np.ones(len(genoma_inicial))) for _ in range(num_portfolios - 1)]
    melhor_portfolio = genoma_inicial
    melhor_fitness = fitness_com_penalidade(genoma_inicial, retornos, riscos, taxa_livre_risco)

    for geracao in range(geracoes):
        # Calcular o fitness para cada portfólio
        fitness_scores = np.array([fitness_com_penalidade(port, retornos, riscos, taxa_livre_risco) for port in populacao])

        # Identificar o melhor portfólio
        indice_melhor_portfolio = np.argmax(fitness_scores)
        if fitness_scores[indice_melhor_portfolio] > melhor_fitness:
            melhor_fitness = fitness_scores[indice_melhor_portfolio]
            melhor_portfolio = populacao[indice_melhor_portfolio]

        # Seleção e cruzamento (crossover) e mutação
        populacao = selecao_torneio(populacao, fitness_scores)
        nova_populacao = []
        for i in range(0, len(populacao), 2):
            pai1, pai2 = populacao[i], populacao[i+1]

            filho1, filho2 = cruzamento_multiponto(pai1, pai2)
            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        populacao = nova_populacao

        # Exibir o melhor Sharpe Ratio da geração atual
        print(f"Geracao {geracao + 1}, Melhor Sharpe Ratio: {melhor_fitness}")

    return melhor_portfolio

# Função de cruzamento de múltiplos pontos
def cruzamento_multiponto(pai1, pai2, num_pontos=3):
    # Garantir que o número de pontos de corte seja válido e não maior que o comprimento dos pais
    num_pontos = min(num_pontos, len(pai1) - 1)

    # Gerar pontos de corte de forma aleatória
    pontos_corte = sorted(np.random.choice(range(1, len(pai1)), num_pontos, replace=False))

    # Adicionar início e fim para garantir cruzamento completo
    pontos_corte = [0] + pontos_corte + [len(pai1)]

    filho1, filho2 = pai1.copy(), pai2.copy()
    for i in range(0, len(pontos_corte) - 1, 2):
        filho1[pontos_corte[i]:pontos_corte[i + 1]] = pai2[pontos_corte[i]:pontos_corte[i + 1]]
        filho2[pontos_corte[i]:pontos_corte[i + 1]] = pai1[pontos_corte[i]:pontos_corte[i + 1]]

    return filho1, filho2

# Função de mutação
def mutacao(portfolio, taxa_mutacao=0.01):
    if np.random.random() < taxa_mutacao:
        i = np.random.randint(0, len(portfolio))
        portfolio[i] += np.random.uniform(-0.1, 0.1)
        portfolio = np.clip(portfolio, 0, 1)  # Manter alocações válidas
        portfolio /= portfolio.sum()  # Normalizar
    return portfolio

# Funções auxiliares: seleção por torneio
def selecao_torneio(populacao, fitness_scores, tamanho_torneio=3):
    selecionados = []
    for _ in range(len(populacao)):
        competidores = np.random.choice(len(populacao), tamanho_torneio, replace=False)
        vencedor = competidores[np.argmax(fitness_scores[competidores])]
        selecionados.append(populacao[vencedor])
    return selecionados

# Exemplo de dados reais para retornos e riscos
retornos_reais = np.random.rand(34) * 0.2  # Retornos simulados entre 0 e 20%
riscos_reais = np.random.rand(34) * 0.1    # Riscos simulados entre 0 e 10%

# Rodar o algoritmo genético com penalidade
melhor_portfolio = algoritmo_genetico_com_penalidade(retornos_reais, riscos_reais, genoma_inicial)

# Distribuir 100 mil reais entre os ativos com base na melhor alocação
total_investido = 100000  # 100 mil reais
distribuicao_investimento = melhor_portfolio * total_investido

# Criar um DataFrame para exibir a distribuição
ativos = df['Ativo'].values  # Lista dos ativos
distribuicao_df = pd.DataFrame({
    'Ativo': ativos,
    'Alocacao (%)': melhor_portfolio * 100,
    'Valor Investido (R$)': distribuicao_investimento
})

# Exibir a distribuição ideal do investimento
print(distribuicao_df)


[*********************100%***********************]  15 of 15 completed


Geracao 1, Melhor Sharpe Ratio: 0.20295757130620848
Geracao 2, Melhor Sharpe Ratio: 0.38671232417367224
Geracao 3, Melhor Sharpe Ratio: 0.6384264322742438
Geracao 4, Melhor Sharpe Ratio: 0.7331034639008677
Geracao 5, Melhor Sharpe Ratio: 0.9615481532097634
Geracao 6, Melhor Sharpe Ratio: 1.094955351155603
Geracao 7, Melhor Sharpe Ratio: 1.3107026414527962
Geracao 8, Melhor Sharpe Ratio: 1.433328629185364
Geracao 9, Melhor Sharpe Ratio: 1.6391889864149025
Geracao 10, Melhor Sharpe Ratio: 1.8847999273619813
Geracao 11, Melhor Sharpe Ratio: 2.006305959521261
Geracao 12, Melhor Sharpe Ratio: 2.1557853127267754
Geracao 13, Melhor Sharpe Ratio: 2.1557853127267754
Geracao 14, Melhor Sharpe Ratio: 2.1673357632561845
Geracao 15, Melhor Sharpe Ratio: 2.2283867568423217
Geracao 16, Melhor Sharpe Ratio: 2.316674126321744
Geracao 17, Melhor Sharpe Ratio: 2.3512112228168807
Geracao 18, Melhor Sharpe Ratio: 2.3512112228168807
Geracao 19, Melhor Sharpe Ratio: 2.3780404160745365
Geracao 20, Melhor Shar

# Usando o Streamlit

##Passamos o código com o crossover de ponto único, geralmente mais usado, para o Github e trabalhamos o Streamlit direto nele.